<a href="https://colab.research.google.com/github/safal207/hello-world/blob/master/%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%B3%D0%B8%D0%BF%D0%B5%D1%80%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D0%BE%D0%B2_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D0%B8_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_Keras_Tuner_%22fashion_mnist_keras_tuner_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оптимизация гиперпараметров нейросети с помощью [Keras Tuner](https://github.com/keras-team/keras-tuner)



Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

Не забудьте подключить GPU, чтобы сеть обучалась быстрее (Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU).



## Гиперпараметры обучения нейронной сети

- Количество слоев нейронной сети
- Количество нейронов в каждом слое
- Функции активации, которые используются в слоях
- Тип оптимизатора при обучении нейронной сети
- Количество эпох обучения

## Установка Keras Tuner

In [1]:
pip install -U keras-tuner

     |████████████████████████████████| 61kB 6.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=3baddec98c348aff176b77c3564604430411ad676499232ba2e7c2eceacfd7bb
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=ecaaf823799fb00909d207f59b65b235fe81174d386163811d3394a48547e8ea
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


## Подключаем нужные пакеты

In [2]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from google.colab import files
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

## Подготовка данных для обучения сети

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255 
x_test = x_test / 255 
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

## Задаем функцию создания нейронной сети

In [5]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 128
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation=activation_choice))   
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Создаем tuner

Доступные типы тюнеров: 
- RandomSearch - случайный поиск.
- Hyperband - алгоритм оптимизации на основе многорукого бандита, Li, Lisha, and Kevin Jamieson. ["Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization."Journal of Machine Learning Research 18 (2018): 1-52](http://jmlr.org/papers/v18/16-558.html).
- BayesianOptimization - [байесовская оптимизация](https://en.wikipedia.org/wiki/Bayesian_optimization).

In [6]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=80,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )

## Запускаем подбор гиперпараметров

Пространство поиска

In [7]:
tuner.search_space_summary()

Подбор гиперпараметров

In [8]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=20,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Epoch 1/20
188/188 [==============================] - 1s 5ms/step - loss: 0.6416 - accuracy: 0.7667 - val_loss: 0.4312 - val_accuracy: 0.8430
Epoch 2/20
188/188 [==============================] - 1s 5ms/step - loss: 0.4165 - accuracy: 0.8455 - val_loss: 0.4039 - val_accuracy: 0.8517
Epoch 3/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3607 - accuracy: 0.8663 - val_loss: 0.3891 - val_accuracy: 0.8564
Epoch 4/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3275 - accuracy: 0.8785 - val_loss: 0.3238 - val_accuracy: 0.8811
Epoch 5/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3012 - accuracy: 0.8890 - val_loss: 0.3371 - val_accuracy: 0.8799
Epoch 6/20
188/188 [==============================] - 1s 5ms/step - loss: 0.2868 - accuracy: 0.8912 - val_loss: 0.3207 - val_accuracy: 0.8842
Epoch 7/20
188/188 [==============================] - 1s 4ms/step - loss: 0.2651 - accuracy: 0.8995 - val_loss: 0.3588 - val_accuracy: 0.8767
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 5ms/step - loss: 0.5946 - accuracy: 0.8038 - val_loss: 0.4248 - val_accuracy: 0.8497
Epoch 2/20
188/188 [==============================] - 1s 4ms/step - loss: 0.4185 - accuracy: 0.8489 - val_loss: 0.3888 - val_accuracy: 0.8607
Epoch 3/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3666 - accuracy: 0.8668 - val_loss: 0.3649 - val_accuracy: 0.8663
Epoch 4/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3468 - accuracy: 0.8727 - val_loss: 0.4057 - val_accuracy: 0.8514
Epoch 5/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3271 - accuracy: 0.8804 - val_loss: 0.3878 - val_accuracy: 0.8633
Epoch 6/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3071 - accuracy: 0.8867 - val_loss: 0.3719 - val_accuracy: 0.8702
Epoch 7/20
188/188 [==============================] - 1s 4ms/step - loss: 0.2950 - accuracy: 0.8918 - val_loss: 0.3528 - val_accuracy: 0.8749
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 5ms/step - loss: 0.5308 - accuracy: 0.8092 - val_loss: 0.4480 - val_accuracy: 0.8363
Epoch 2/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3940 - accuracy: 0.8571 - val_loss: 0.4131 - val_accuracy: 0.8479
Epoch 3/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3523 - accuracy: 0.8709 - val_loss: 0.3677 - val_accuracy: 0.8677
Epoch 4/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3272 - accuracy: 0.8794 - val_loss: 0.3457 - val_accuracy: 0.8722
Epoch 5/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3070 - accuracy: 0.8862 - val_loss: 0.3885 - val_accuracy: 0.8590
Epoch 6/20
188/188 [==============================] - 1s 4ms/step - loss: 0.2926 - accuracy: 0.8902 - val_loss: 0.3234 - val_accuracy: 0.8837
Epoch 7/20
188/188 [==============================] - 1s 5ms/step - loss: 0.2717 - accuracy: 0.8991 - val_loss: 0.3275 - val_accuracy: 0.8802
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 5ms/step - loss: 0.7785 - accuracy: 0.7283 - val_loss: 0.5088 - val_accuracy: 0.8110
Epoch 2/20
188/188 [==============================] - 1s 5ms/step - loss: 0.4570 - accuracy: 0.8348 - val_loss: 0.4246 - val_accuracy: 0.8457
Epoch 3/20
188/188 [==============================] - 1s 5ms/step - loss: 0.4113 - accuracy: 0.8526 - val_loss: 0.4071 - val_accuracy: 0.8523
Epoch 4/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3802 - accuracy: 0.8632 - val_loss: 0.3850 - val_accuracy: 0.8611
Epoch 5/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3634 - accuracy: 0.8688 - val_loss: 0.3795 - val_accuracy: 0.8602
Epoch 6/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3476 - accuracy: 0.8733 - val_loss: 0.3648 - val_accuracy: 0.8648
Epoch 7/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3350 - accuracy: 0.8781 - val_loss: 0.3583 - val_accuracy: 0.8713
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 5ms/step - loss: 0.5135 - accuracy: 0.8192 - val_loss: 0.4804 - val_accuracy: 0.8150
Epoch 2/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3656 - accuracy: 0.8654 - val_loss: 0.3629 - val_accuracy: 0.8677
Epoch 3/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3206 - accuracy: 0.8811 - val_loss: 0.3285 - val_accuracy: 0.8812
Epoch 4/20
188/188 [==============================] - 1s 5ms/step - loss: 0.2933 - accuracy: 0.8917 - val_loss: 0.3225 - val_accuracy: 0.8832
Epoch 5/20
188/188 [==============================] - 1s 5ms/step - loss: 0.2734 - accuracy: 0.8985 - val_loss: 0.3367 - val_accuracy: 0.8760
Epoch 6/20
188/188 [==============================] - 1s 5ms/step - loss: 0.2614 - accuracy: 0.9031 - val_loss: 0.3236 - val_accuracy: 0.8826
Epoch 7/20
188/188 [==============================] - 1s 5ms/step - loss: 0.2419 - accuracy: 0.9098 - val_loss: 0.3095 - val_accuracy: 0.8873
Epoch 

...

Epoch 1/20
188/188 [==============================] - 1s 4ms/step - loss: 0.8691 - accuracy: 0.7185 - val_loss: 0.6415 - val_accuracy: 0.7763
Epoch 2/20
188/188 [==============================] - 1s 4ms/step - loss: 0.5872 - accuracy: 0.8025 - val_loss: 0.5500 - val_accuracy: 0.8112
Epoch 3/20
188/188 [==============================] - 1s 4ms/step - loss: 0.5288 - accuracy: 0.8205 - val_loss: 0.5194 - val_accuracy: 0.8134
Epoch 4/20
188/188 [==============================] - 1s 4ms/step - loss: 0.4987 - accuracy: 0.8289 - val_loss: 0.4939 - val_accuracy: 0.8251
Epoch 5/20
188/188 [==============================] - 1s 4ms/step - loss: 0.4789 - accuracy: 0.8356 - val_loss: 0.4829 - val_accuracy: 0.8289
Epoch 6/20
188/188 [==============================] - 1s 4ms/step - loss: 0.4643 - accuracy: 0.8413 - val_loss: 0.4823 - val_accuracy: 0.8257
Epoch 7/20
188/188 [==============================] - 1s 4ms/step - loss: 0.4530 - accuracy: 0.8430 - val_loss: 0.4603 - val_accuracy: 0.8358
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 5ms/step - loss: 0.6177 - accuracy: 0.8009 - val_loss: 0.4633 - val_accuracy: 0.8332
Epoch 2/20
188/188 [==============================] - 1s 5ms/step - loss: 0.4104 - accuracy: 0.8524 - val_loss: 0.4105 - val_accuracy: 0.8556
Epoch 3/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3728 - accuracy: 0.8643 - val_loss: 0.4477 - val_accuracy: 0.8353
Epoch 4/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3481 - accuracy: 0.8712 - val_loss: 0.3678 - val_accuracy: 0.8687
Epoch 5/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3203 - accuracy: 0.8816 - val_loss: 0.3526 - val_accuracy: 0.8720
Epoch 6/20
188/188 [==============================] - 1s 4ms/step - loss: 0.3099 - accuracy: 0.8852 - val_loss: 0.3795 - val_accuracy: 0.8602
Epoch 7/20
188/188 [==============================] - 1s 4ms/step - loss: 0.2917 - accuracy: 0.8916 - val_loss: 0.3450 - val_accuracy: 0.8769
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 5ms/step - loss: 0.7519 - accuracy: 0.7341 - val_loss: 0.6347 - val_accuracy: 0.7593
Epoch 2/20
188/188 [==============================] - 1s 4ms/step - loss: 0.4719 - accuracy: 0.8234 - val_loss: 0.4232 - val_accuracy: 0.8416
Epoch 3/20
188/188 [==============================] - 1s 5ms/step - loss: 0.4048 - accuracy: 0.8479 - val_loss: 0.4529 - val_accuracy: 0.8444
Epoch 4/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3585 - accuracy: 0.8647 - val_loss: 0.3890 - val_accuracy: 0.8528
Epoch 5/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3322 - accuracy: 0.8751 - val_loss: 0.3795 - val_accuracy: 0.8622
Epoch 6/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3107 - accuracy: 0.8821 - val_loss: 0.3834 - val_accuracy: 0.8589
Epoch 7/20
188/188 [==============================] - 1s 5ms/step - loss: 0.2905 - accuracy: 0.8895 - val_loss: 0.3440 - val_accuracy: 0.8723
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 5ms/step - loss: 0.5200 - accuracy: 0.8150 - val_loss: 0.3897 - val_accuracy: 0.8569
Epoch 2/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3611 - accuracy: 0.8684 - val_loss: 0.3638 - val_accuracy: 0.8706
Epoch 3/20
188/188 [==============================] - 1s 5ms/step - loss: 0.3196 - accuracy: 0.8825 - val_loss: 0.3439 - val_accuracy: 0.8733
Epoch 4/20
188/188 [==============================] - 1s 5ms/step - loss: 0.2980 - accuracy: 0.8902 - val_loss: 0.3256 - val_accuracy: 0.8797
Epoch 5/20
188/188 [==============================] - 1s 5ms/step - loss: 0.2744 - accuracy: 0.8975 - val_loss: 0.3193 - val_accuracy: 0.8858
Epoch 6/20
188/188 [==============================] - 1s 5ms/step - loss: 0.2619 - accuracy: 0.9022 - val_loss: 0.3220 - val_accuracy: 0.8829
Epoch 7/20
188/188 [==============================] - 1s 5ms/step - loss: 0.2495 - accuracy: 0.9066 - val_loss: 0.3077 - val_accuracy: 0.8898
Epoch 

INFO:tensorflow:Oracle triggered exit


## Выбираем лучшую модель

In [9]:
tuner.results_summary()

Получаем три лучших модели

In [10]:
models = tuner.get_best_models(num_models=3)

Оцениваем качество модели на тестовых данных

In [11]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 704)               552640    
_________________________________________________________________
dense_1 (Dense)              (None, 320)               225600    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                3210      
Total params: 781,450
Trainable params: 781,450
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 1s 2ms/step - loss: 0.3491 - accuracy: 0.8911

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 832)               653120    
_________________________________________________________________
dense_